# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [ ]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2021-04-17 00:59:24--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  8.76MB/s    in 0.5s    

2021-04-17 00:59:24 (8.76 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [ ]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [ ]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.832747220993042),
 ('prince', 0.7576971054077148),
 ('york', 0.7520603537559509),
 ('warwick', 0.7433580756187439),
 ('duke', 0.7370203733444214),
 ('clarence', 0.7252764701843262),
 ('princess', 0.7183069586753845),
 ('gloucester', 0.6981997489929199),
 ('cardinal', 0.685215950012207),
 ('northumberland', 0.6812140941619873)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [ ]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9656510353088379),
 ('emilia', 0.9320991039276123),
 ('ham', 0.9263814687728882),
 ('cleopatra', 0.9227249622344971),
 ('cassio', 0.920681357383728),
 ('iago', 0.9193363785743713),
 ('helena', 0.9066823720932007),
 ('pisanio', 0.9065233469009399),
 ('valentine', 0.9025164842605591),
 ('imogen', 0.8993434906005859)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [ ]:
model.wv.similarity('othello', 'desdemona')

0.965651

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
import sklearn.metrics as metrics
import math
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  v1 = v1.reshape(1,-1)
  v2 = v2.reshape(1,-1)
  dot_prod = np.dot(v1,v2.T)
  norm_prod = np.linalg.norm(v1)*np.linalg.norm(v2)

  return dot_prod/norm_prod

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

array([[0.965651]], dtype=float32)

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2021-04-17 03:26:18--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.2’

30k.en.vec.2        100%[===================>]  64.54M  31.5MB/s    in 2.0s    

2021-04-17 03:26:21 (31.5 MB/s) - ‘30k.en.vec.2’ saved [67681172/67681172]

--2021-04-17 03:26:21--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.2’

30k.fr.vec.2        100%[===================>]  64.66M  33.9MB/s    in 1.9s    

2021-04-17 03:26:23 (33.9 MB/s) - ‘30k.fr.vec.2’ saved [67802327/67802327]

--2021-04-17 03:26:23--  http://www.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [ ]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [ ]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0

# Create an empty dictionary to store the similarity values of each vector
  cosim_arr = {}
  for i in vecDict.keys():
    cosim_arr[i]=cosim(vec,vecDict[i])

# Get the maximum similarity
  similarity = max(cosim_arr.values())

# Extract the word with maximum similarity
  for k,v in cosim_arr.items():
    if v == similarity:
      mostSimilar = k   

  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', array([[0.50238278]])),
 ('allemagne', array([[0.59371841]])),
 ('matrice', array([[0.50883613]])),
 ('physique', array([[0.45555434]])),
 ('fermentation', array([[0.35041052]]))]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2021-04-17 03:28:07--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.2’

links.tab.2         100%[===================>]   1.34M  3.50MB/s    in 0.4s    

2021-04-17 03:28:08 (3.50 MB/s) - ‘links.tab.2’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [ ]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [ ]:
def evaluate_models(lang,langvec):
  accuracy = 0
  baselineAccuracy = 0
  article_total = 0 
  match_count = 0
  basematch_count =0 

  for l in links:
    if l[1]== lang:
      article_total += 1
      if l[0] in envec.keys() and mostSimilar(envec[l[0]],langvec)[0]== l[2]:
        match_count += 1
      if l[0] in envec.keys() and l[0]==l[2]:
        basematch_count += 1
      else:
        continue

  acc = match_count / article_total 
  base_acc = basematch_count / article_total
  
  return acc, base_acc

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
eng_acc, eng_baseline_acc = evaluate_models('fr',frvec)
print(" Accuracy of English-French Wikipedia is ", eng_acc)
print("Baseline accuracy of English-French Wikipedia is ", eng_baseline_acc)

 Accuracy of English-French Wikipedia is  0.5759236593278961
Baseline accuracy of English-French Wikipedia is  0.6442534558227827


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
es_acc, es_baseline_acc = evaluate_models('es',esvec)
print(" Accuracy of English-Espanol Wikipedia is ", es_acc)
print("Baseline accuracy of English-Espanol Wikipedia is ", es_baseline_acc)

 Accuracy of English-Espanol Wikipedia is  0.0759319427255674
Baseline accuracy of English-Espanol Wikipedia is  0.2276587239848372


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".